# Using FABRIC GPUs

Your compute nodes can include GPUs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use GPU devices on FABRIC.

## Configure the Environment

In [ ]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE'] = ''

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create a Node

The cell below creates a slice that contains a single node. The node includes a GPU component.

### Set the Slice Name and FABRIC Site

In [ ]:
slice_name="MySliceGPU"
site="MAX"
node_name='Node1'
image = 'default_ubuntu_20'
cores = 2
ram = 8
disk = 10

gpu_name='gpu1'

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=cores, ram=ram, disk=disk)
    node.set_image(image)
    
    #Add an NVME Drive
    #node.add_component(model='GPU_TeslaT4', name=gpu_name)
    # OR
    node.add_component(model='GPU_RTX6000', name=gpu_name)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

## Get the Slice

Retrieve the node information and save the management IP addresses.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slice Error: {e}")

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
try:
    node = slice.get_node(node_name) 
    print(f"Node Name        : {node.get_name()}")
    print(f"Management IP    : {node.get_management_ip()}")
    print(f"SSH Command      : {node.get_ssh_command()}")
    print()

    gpu1 = node.get_component(gpu_name)
    print(f"GPU Name         : {gpu1.get_name()}")
    print(f"Details          : {gpu1.get_details()}")
    print(f"Units            : {gpu1.get_unit()}")
    print(f"PCI Address      : {gpu1.get_pci_addr()}")
    print(f"Model            : {gpu1.get_model()}")
    print(f"Type             : {gpu1.get_type()}")  
except Exception as e:
    print(f"Error: {e}")



### GPU PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

An example of using the GPUs is coming soon.

View node1's GPU

In [ ]:
command = 'lspci'
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Fail: {e}")

## Cleanup Your Experiment

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()